In [1]:
#imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


### Parte 1 - Leitura de Dataset

A leitura do Dataset foi realizada utilizando o Pandas. A base é uma base que analisa se 
vai jogar ou não tênis baseado nos atributos: tempo, temperatura, umidade e vento.

In [2]:
data = [['Sol'    , 'quente', 'alta', 'Não', 'Não'],
        ['Sol'    , 'quente', 'alta', 'Sim', 'Não'],
        ['Nublado', 'quente', 'alta', 'Não', 'Sim'],
        ['Chuva'  , 'moderada', 'alta', 'Não', 'Sim'],
        ['Chuva'  , 'fria', 'normal', 'Não', 'Sim'],
        ['Chuva'  , 'fria', 'normal', 'Sim', 'Não'],
        ['Nublado', 'fria', 'normal', 'Sim', 'Sim'],
        ['Sol'    , 'moderada', 'alta', 'Não', 'Não'],
        ['Sol'    , 'fria', 'normal', 'Não', 'Sim'],
        ['Chuva'  , 'moderada', 'normal', 'Não', 'Sim'],
        ['Sol'    , 'moderada', 'normal', 'Sim', 'Sim'],
        ['Nublado', 'moderada', 'alta', 'Sim', 'Sim'],
        ['Nublado', 'quente', 'normal', 'Não', 'Sim'],
        ['Chuva'  , 'moderada', 'alta', 'Sim', 'Não'],
       ];

df = pd.DataFrame(data)

In [3]:
df

,0,1,2,3,4
0,Sol,quente,alta,Não,Não
1,Sol,quente,alta,Sim,Não
2,Nublado,quente,alta,Não,Sim
3,Chuva,moderada,alta,Não,Sim
4,Chuva,fria,normal,Não,Sim
5,Chuva,fria,normal,Sim,Não
6,Nublado,fria,normal,Sim,Sim
7,Sol,moderada,alta,Não,Não
8,Sol,fria,normal,Não,Sim
9,Chuva,moderada,normal,Não,Sim


### Parte 1 - Obter a frequência dos dados

Para cálculo da entropia, deve-se descobrir a frequência do valor analisado.
Para isso, utilizando o Pandas, a frequência é calculada através da condicional do valor do
atributo existir na classe. Esse processo é repetido para cada classe até que todas as frequências
sejam obtidas

In [4]:
def getFrequency(data, classNames):
    """Funçaõ que calcula a frequencia dos atributos, recebe como entrada o dataframe
    e o índice da coluna do atributo alvo.
    Deve-se verificar se o dataframe existe, retornando frequencia 0 caso contrário.
    Retorno: array de frequencias de cada classe para um dataframe
    """
    frequency = []
    for i in range(len(classNames)):
        if len(data)==0:
            frequency.append(0)
        else:
            frequency.append(data.iloc[:,data.shape[1]-1][data.iloc[:,data.shape[1]-1]==classNames[i]].count())
    return frequency

### Parte 2 - Entropia

A árvore de decisão é baseada no ganho de informação e entropia. Dessa forma, para entropia,
que define o grau de impureza da base de dados analisada (ou partição da mesma), calculada
a partir de duas condições:
    
* se os atributos estiverem com mesma proporção para todas as classes de saída, 
a entropia é 1 (máxima).
* se uma classe obtiver todos as instâncias, a entropia é 0 (mínima)
* caso nenhum desses casos aconteça, a entropia deve ser calculada através da equação:
    
    $$ Entropia = - entropiaDaPartição * \sum{log2(frequenciaDdeClasseX/totalDeInstancias)} $$



In [5]:
def entropy(positive,negative):
    total = positive+negative
    if positive == negative:
        return 1
    elif negative == 0 or positive==0:
        return 0
    else: 
        return -(positive/total)*np.log2(positive/total) - (negative/total)*np.log2(negative/total);

### Parte 3 - Dividir o dataset

Para cada atributo deve-se dividir o dataset em n partições, onde n é o número de 
classes do problema (2 classes para esse caso)

In [6]:
def divideset(data, column, value):
    split_function = lambda row:row[column] == value
        
    set1 = [row for row in data if split_function(row)]
    set2 = [row for row in data if not split_function(row)]
    
    #converte para dataframe
    df1 = pd.DataFrame(set1)
    df2 = pd.DataFrame(set2)
    return (df1, df2)

### Parte 5 - Estrutura da Árvore

A árvore possui cinco nós: 

* o nó com o resultado final do atributo
* Regra associada ao nó
* Coluna do atributo
* Os ramos do nó


In [7]:
class DecisionNode:
    def __init__(self, col=-1, value=None, results=None, tb=None, fb=None):
        self.col = col          # indice da coluna do criterio a ser testado
        self.value = value      # valor que deve satisfazer para conseguir um resultado verdadeiro
        self.results = results  # armazena um dicionario dos resultado para esse ramo. None se o nó de decisão for folha.
        self.tb = tb            # proximos nó na arvore se o resultado é true
        self.fb = fb            # proximos nó na arvore se o resultado é false

In [8]:
def tree(data):
    if len(data)==0: #chegou na folha
        return DecisionNode() #fim da árvore
    best_gain,best_criteria, best_sets = 0.0, None, None
    f = getFrequency(data, df.iloc[:,4].unique()) #calcula a frequencia do dataframe atual
    current = entropy(f[0], f[1]) #calcula o valor da entropia atual
    for col in range(data.shape[1]-1): #para cada atributo
        # print('FIM DO SET')
        gain = []
        for row in range(data.shape[0]):
            #divide o dataset baseado nas classes de saída
            (set1,set2)  = divideset(data.values,col,data.iloc[row,col]) 
            p = float((len(set1)))/data.shape[0] #calcula a probabilidade do atributo
            #calcula a frequencia de cada partição
            f1 = getFrequency(set1,df.iloc[:,4].unique()) 
            f2 = getFrequency(set2, df.iloc[:,4].unique())
            #calcula o ganho a partir das frequencias e da entropia calculadas
            #para o dataframe
            gain.append(current -p*entropy(f1[0],f1[1]) -(1-p)*entropy(f2[0],f2[1]))
            g = np.max(gain) #retorna o maior ganho
            if g > best_gain and len(set1) > 0 and len(set2) > 0: #se esse ganho é o maior 
                best_gain = g
                best_criteria = (col, data.iloc[row,col]) #armazena valor do nó
                best_sets = (set1,set2) #guarda as duas partiçõas dos ramos da árvore
    if best_gain > 0:
        true_branch = tree(best_sets[0])
        false_branch = tree(best_sets[1])
        return DecisionNode(col=best_criteria[0], value=best_criteria[1], tb=true_branch, fb=false_branch)
    else:
        return DecisionNode(results=f)
        

In [9]:
def printtree(tree, indent=''):
    if tree.results != None:
        print(tree.results)
    else:
        print('{0}:{1}?'.format(tree.col, tree.value))
        print(indent+"-", end='')
        printtree(tree.tb, indent + '  ')
        print(indent+"-", end='')
        printtree(tree.fb, indent + '  ')

In [10]:
def classify(tree, sample):
    if tree.results != None: return tree.results
    else:
        v = sample[tree.col]
        branch = None
        branch = tree.tb if v == tree.value else tree.fb
        return classify(branch, sample)

In [11]:
X_train, X_test = train_test_split(df, test_size=0.25,random_state=0)

In [12]:
t = tree(X_train)

In [13]:
printtree(t)

0:Nublado?
-[0, 2]
-1:moderada?
  -2:alta?
    -0:Chuva?
      -3:Sim?
        -[1, 0]
        -[0, 1]
      -[1, 0]
    -[0, 2]
  -[3, 0]


In [14]:
print(classify(t, X_test.iloc[1,:]))

[0, 2]
